Install MMDet

In [1]:
!pip3 install openmim --quiet
!mim install mmengine --quiet
!mim install "mmcv>=2.0.0rc1,<2.1.0" --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 895.5 kB/s eta 0:00:00
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.9/366.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 16.1 MB/s eta 0:00:00
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and

In [2]:
!git clone https://github.com/open-mmlab/mmdetection.git -b 3.x --quiet
%cd mmdetection
!pip install -e . --quiet

/content/mmdetection
  Preparing metadata (setup.py) ... done


In [ ]:
import mmdet
print(mmdet.__version__)
# Example output: 3.0.0rc0, or an another version.

3.0.0


### Load Checkpoint File and Config File

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/rtmdet_tiny.py /content/.

In [5]:
!cp /content/drive/MyDrive/rtmdet-ins_tiny_8xb32-300e_coco_20221130_151727-ec670f7e.pth /content/.

### Modify the Config File

1. Set path to base config
2. Set number of Classes

All this should already be done in the provided config file.

## Check if the Model works

In [ ]:
from mmdet.apis import init_detector, inference_detector
import mmcv
import numpy as np
import time


# Specify the path to model config and checkpoint file
config_file = '/content/rtmdet_tiny.py'
checkpoint_file = '/content/rtmdet-ins_tiny_8xb32-300e_coco_20221130_151727-ec670f7e.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cpu') #cpu

Loads checkpoint by local backend from path: /content/rtmdet-ins_tiny_8xb32-300e_coco_20221130_151727-ec670f7e.pth


#### Convert to Openvino

In [ ]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 54.5 MB/s eta 0:00:00


In [ ]:
!pip install openvino

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 36.8 MB/s eta 0:00:00


In [ ]:
import sys
import time
import warnings
from pathlib import Path

import cv2
import numpy as np
import torch
from IPython.display import Markdown, display
from torchvision.models.segmentation import lraspp_mobilenet_v3_large, LRASPP_MobileNet_V3_Large_Weights
from openvino.runtime import Core

sys.path.append("../utils")
#from notebook_utils import segmentation_map_to_image, viz_result_image, SegmentationMap, Label, download_file

In [ ]:
IMAGE_WIDTH = 640
IMAGE_HEIGHT = 640

# Paths where ONNX and OpenVINO IR models will be stored.
onnx_path = Path('segmentation_onnx.onnx')
if not onnx_path.parent.exists():
    onnx_path.parent.mkdir()
ir_path = onnx_path.with_suffix(".xml")

/content


In [ ]:
model.to("cpu")

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    if not onnx_path.exists():
        dummy_input = torch.randn(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH)
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
        )
        print(f"ONNX model exported to {onnx_path}.")
    else:
        print(f"ONNX model {onnx_path} already exists.")

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX model exported to segmentation_onnx.onnx.


Add extension to replace hardsigmoid

In [ ]:
from openvino.runtime import Core
from openvino.frontend import ConversionExtension
import openvino.runtime.opset8 as ops


In [ ]:
ov = Core()
from openvino.runtime.opset8 import multiply, add, constant

def conversion(node):
    alpha = 0.2
    beta = 0.5
    x = node.get_input(0)
    alpha_const = constant(alpha, x.get_element_type(), x.get_shape())
    beta_const = constant(beta, x.get_element_type(), x.get_shape())
    hard_sigmoid_output = ops.clamp(add(multiply(x, alpha_const), beta_const), 0, 1).outputs()
    return hard_sigmoid_output


In [ ]:
ov.add_extension(ConversionExtension('HardSigmoid', conversion))


In [ ]:
converted = ov.read_model(onnx_path)
converted

<Model: 'torch_jit'
inputs[
<ConstOutput: names[input.1] shape[1,3,640,640] type: f32>
]
outputs[
<ConstOutput: names[867] shape[1,80,80,80] type: f32>,
<ConstOutput: names[897] shape[1,80,40,40] type: f32>,
<ConstOutput: names[927] shape[1,80,20,20] type: f32>,
<ConstOutput: names[888] shape[1,4,80,80] type: f32>,
<ConstOutput: names[918] shape[1,4,40,40] type: f32>,
<ConstOutput: names[948] shape[1,4,20,20] type: f32>,
<ConstOutput: names[876] shape[1,169,80,80] type: f32>,
<ConstOutput: names[906] shape[1,169,40,40] type: f32>,
<ConstOutput: names[936] shape[1,169,20,20] type: f32>,
<ConstOutput: names[858] shape[1,8,80,80] type: f32>
]>

In [ ]:
from openvino.runtime import serialize


In [ ]:
serialize(converted, xml_path="segmentation_converted.xml")


https://blobconverter.luxonis.com/
